In [1]:
### Probando con datos
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

conf = (SparkConf().set("spark.executor.memory", "6g")   #<--- Por executor. Como hay dos instancias, la mitad por executor.
                    .set("spark.executor.cores", "3")    #<--- Por cada executor. Como hay 4, 4*3=12 cores se usan.
                    .set("spark.executor.instances", "2")
                    .set("spark.eventLog.enabled", "true"))
spark = SparkSession.builder.master("spark://spark-master:7077").appName("ReadParquetYearMon16MM").config(conf=conf).getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/16 18:00:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import time
print ('Inicio: '+time.strftime("%c"))
inicio = time.perf_counter()

Inicio: Mon Jan 16 18:00:46 2023


In [3]:
%%time
#spark job monitoring
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.sql.functions import sum, col, desc

esquema = StructType() \
      .add("Id",StringType(),True) \
      .add("fecNac",StringType(),True) \
      .add("sex",StringType(),True) \
      .add("ingresos",IntegerType(),True) \
      .add("partido",StringType(),True) \
      .add("edad",IntegerType(),True) \
      .add("mapa",StringType(),True)

#df = spark.read.parquet("./datos/fakePeopleParallel100M.parquet", sep=";", header = True, schema = esquema)

#df = spark.read.parquet("./datos/more/fakePeopleParallel100M.parquet/sex=*/partido=PP", sep=";", header = True, schema = esquema)

#df = spark.read.option("header", "false").option("schema", "esquema").option("mergeSchema", "true").option("sep", ";").parquet("./datos/more/fakePeopleParallel100M.parquet/sex=*/partido=P*")

"""
 * Lectura de un rango de meses de un parquet particionado por año/mes:
"""

def listPartitions (anoMesFinal, meses, path):
    lista = []
    mes = int(anoMesFinal[-2:]) + 1
    ano = int(anoMesFinal[:-3])
    
    print('Primer mes: '+anoMesFinal)
    
    for i in range(meses):
        mes = mes - 1
        if mes == 0:
            ano = ano - 1
            mes = 12
        if mes < 10: anoMes = str(ano)+'-0'+str(mes)
        else: anoMes = str(ano)+'-'+str(mes)
        #print('Desde la función: '+str(i+1)+' - '+anoMes)

        lista.append(path+anoMes)
    print('Último mes: '+anoMes)
    return lista

#df = spark.read.option("mergeSchema", True).option("schema", "esquema").opt/opt/workspace/datos/fecNac/fakePeople.parquetion("basePath", "file:///opt/workspace/datos/fecNac/fakePeople.parquet/").parquet('./datos/fecNac/fakePeople.parquet/yearMon=*')

#df = spark.read.option("mergeSchema", True).option("schema", "esquema").option("basePath", "file:///opt/workspace/datos/fecNac/fakePeople.parquet/")\
#               .format("parquet").load('./datos/fecNac/fakePeople.parquet/yearMon=1962-1*')


df = spark.read.option("mergeSchema", True).option("schema", "esquema").option("basePath", "file:///opt/workspace/datos/fecNac/fakePeople1600M.parquet/")\
               .format("parquet").load(listPartitions('2003-12', 3*12, './datos/fecNac/fakePeople1600M.parquet/yearMon='))
                                                                                                   
df.printSchema()

#df = spark.read.format('parquet').load('./datos/more/fakePeopleParallel100M.parquet/')
#df = df.where('sex=H')
print(df.count())
df.show()
df.explain(extended=True)
df.explain(mode='cost')


Primer mes: 2003-12
Último mes: 2001-01


root
 |-- Id: string (nullable = true)
 |-- fecNac: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- ingresos: integer (nullable = true)
 |-- partido: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- mapa: string (nullable = true)
 |-- yearMon: string (nullable = true)



57831664
+------------+----------+---+--------+-------+----+-------------+-------+
|          Id|    fecNac|sex|ingresos|partido|edad|         mapa|yearMon|
+------------+----------+---+--------+-------+----+-------------+-------+
|ID2694381978|2001-10-25|  M|   64400|   PSOE|  20|0-0-0-0-0-0-1|2001-10|
|ID2705876419|2001-10-26|  H|   78000|   Otro|  20|1-1-1-1-0-0-0|2001-10|
|ID2694388738|2001-10-06|  H|   63600|   PSOE|  21|0-0-1-0-0-0-0|2001-10|
|ID2711053790|2001-10-02|  M|   34300|    VOX|  21|1-0-1-0-0-1-1|2001-10|
|ID2711060110|2001-10-28|  H|   27300|     CS|  20|0-0-1-1-0-1-0|2001-10|
|ID2708567699|2001-10-29|  M|   58500|     CS|  20|0-0-1-0-0-1-0|2001-10|
|ID2685314060|2001-10-01|  H|    9600|     CS|  21|0-1-1-0-1-0-0|2001-10|
|ID2697589031|2001-10-20|  M|   28700|     CS|  20|0-0-1-1-0-0-1|2001-10|
|ID2702450821|2001-10-16|  M|   48300|     CS|  20|1-1-1-1-0-0-1|2001-10|
|ID2703510856|2001-10-01|  H|   15300|     CS|  21|1-0-1-0-0-0-0|2001-10|
|ID2697604166|2001-10-09|  H|

In [4]:
print ('Final: '+time.strftime("%c"))
final = time.perf_counter()
print(f'Hecho en {round(final - inicio, 4)} segundo(s)')

Final: Mon Jan 16 18:15:06 2023
Hecho en 860.4021 segundo(s)


In [5]:
print ('Inicio: '+time.strftime("%c"))
inicio = time.perf_counter()

df.groupBy('yearMon').count().sort(desc('yearMon')).show(200)

print ('Final: '+time.strftime("%c"))
final = time.perf_counter()
print(f'Hecho en {round(final - inicio, 4)} segundo(s)')

Inicio: Mon Jan 16 18:15:06 2023


+-------+-------+
|yearMon|  count|
+-------+-------+
|2003-12|1606134|
|2003-11|1606203|
|2003-10|1605319|
|2003-09|1607650|
|2003-08|1606313|
|2003-07|1607364|
|2003-06|1605167|
|2003-05|1607874|
|2003-04|1603991|
|2003-03|1607177|
|2003-02|1605658|
|2003-01|1605350|
|2002-12|1605922|
|2002-11|1607655|
|2002-10|1605743|
|2002-09|1608295|
|2002-08|1606599|
|2002-07|1606351|
|2002-06|1605802|
|2002-05|1607820|
|2002-04|1605569|
|2002-03|1606381|
|2002-02|1604957|
|2002-01|1604904|
|2001-12|1606874|
|2001-11|1606091|
|2001-10|1605899|
|2001-09|1604622|
|2001-08|1609190|
|2001-07|1609491|
|2001-06|1605672|
|2001-05|1607382|
|2001-04|1606146|
|2001-03|1605724|
|2001-02|1606002|
|2001-01|1608373|
+-------+-------+

Final: Mon Jan 16 18:15:24 2023
Hecho en 17.3839 segundo(s)


In [6]:
esqPart = StructType() \
      .add("partido",StringType(),True) \
      .add("nombre",StringType(),True) \
      .add("lider",StringType(),True) \
      .add("fechaCreacion",StringType(),True)

dfPart = spark.read.csv("./datos/partidos.csv", sep=";", header = True, schema = esqPart)

dfPart.show()


+-------+--------------------+--------------------+-------------+
|partido|              nombre|               lider|fechaCreacion|
+-------+--------------------+--------------------+-------------+
|    VOX|     VOX España S.A.|    Santiago Abascal|   2015-01-01|
|     PP|     Partido Popular|Alberto Núñez Feijóo|   1986-10-31|
|   PSOE|Partido Socialist...|       Pedro Sánchez|   1930-12-25|
|     CS|          Ciudadanos|      Inés Arrimadas|   2008-04-18|
|     UP|      Unidas Podemos|        Yolanda Díaz|   2014-04-15|
|  Otros|          Ni se sabe|               Naide|   1975-11-20|
+-------+--------------------+--------------------+-------------+



In [7]:
print ('Inicio: '+time.strftime("%c"))
inicio = time.perf_counter()

df.join(dfPart, df.partido == dfPart.partido, "inner").show()

print ('Final: '+time.strftime("%c"))
final = time.perf_counter()
print(f'Hecho en {round(final - inicio, 4)} segundo(s)')



Inicio: Mon Jan 16 18:15:24 2023
+------------+----------+---+--------+-------+----+-------------+-------+-------+--------------------+--------------------+-------------+
|          Id|    fecNac|sex|ingresos|partido|edad|         mapa|yearMon|partido|              nombre|               lider|fechaCreacion|
+------------+----------+---+--------+-------+----+-------------+-------+-------+--------------------+--------------------+-------------+
|ID2694381978|2001-10-25|  M|   64400|   PSOE|  20|0-0-0-0-0-0-1|2001-10|   PSOE|Partido Socialist...|       Pedro Sánchez|   1930-12-25|
|ID2694388738|2001-10-06|  H|   63600|   PSOE|  21|0-0-1-0-0-0-0|2001-10|   PSOE|Partido Socialist...|       Pedro Sánchez|   1930-12-25|
|ID2711053790|2001-10-02|  M|   34300|    VOX|  21|1-0-1-0-0-1-1|2001-10|    VOX|     VOX España S.A.|    Santiago Abascal|   2015-01-01|
|ID2711060110|2001-10-28|  H|   27300|     CS|  20|0-0-1-1-0-1-0|2001-10|     CS|          Ciudadanos|      Inés Arrimadas|   2008-04-18|
|

### Escribe los datos del join a ver si consigo que dé problema y busco solución

In [8]:
%%time

df.write.mode("overwrite").partitionBy("edad").parquet("./datos/fecNac/fakePeople1600M_join.parquet")

CPU times: user 360 ms, sys: 57.6 ms, total: 418 ms
Wall time: 7min 33s


In [9]:
spark.sparkContext.stop()
print('Sacabao')

Sacabao
